# Summary

This notebook is for data ingestion and carpentry parts of the project. In this notebook we will load the CSV files, subset the data to 2002 - 2019 seasons. We will reformat variables and create new ones. We will create additional player performance metrics that will be used in further analysis. We will also standartize position names and update the historical team name abrevations variables. At the end we will join our data sets into one final data frame we will be using in further analysis.


## Notebook Outline:
- Data description
- Notebook setup
- Loading CSV files
- NBA Sason Stats and Player data
    - cleaning and reformating variables
    - creating new variables
    - joining NBA Season Stats and Player data frames
    - creating NBA Player Per Game
- Team ranking and NBA championship teams data
    - filtering and creating new variables
    - aggregate daily team ranking data frame
    - updating Team name abrevations
- Joining all data frames into one



# Data description


List of CSV files:
- NBA season stats 
- Player data
- Team data
- Daily team ranking and stats 
- NBA champions by season



### NBA season stats 
The data-set contains aggregate individual player statistics for 67 NBA seasons. Variables include from many box-score attributes such as points, assists, rebounds, etc. The data was scraped from Basketball-reference. https://www.basketball-reference.com


##### Data Location
https://www.kaggle.com/drgilermo/nba-players-stats?select=Seasons_Stats.csv


##### Glossary
https://www.basketball-reference.com/about/glossary.html


### Player data
The data-set contains players data. Variables include year start, year end, age, university, etc. 

##### Data Location
https://www.kaggle.com/drgilermo/nba-players-stats?select=player_data.csv


### Team 
The data-set contains Team data. Variables include Team ID, team name, abbreviation of team name, year founded, owner, etc. 

##### Data Location
https://www.kaggle.com/nathanlauga/nba-games?select=teams.csv


### Daily team ranking and stats 
The data-set contains daily team rankings and playing statistics.Variables include season year, games played, games won, win rate, etc. 

##### Data Location
https://www.kaggle.com/nathanlauga/nba-games?select=ranking.csv


### NBA champions by season
The data-set contains NBA champiions teams by season. We have mannualy created this data set. Variables include season, championship team.

##### Data Location
Data source: https://www.basketball-reference.com/playoffs/








# Notebook start


### Setup

In [1]:
# installing additional packages 
install.packages('sqldf')

# loading libraries we will be using
library(tidyr)
library(dplyr)
library(sqldf)

# turning off Factors to load columns as charaters 
options('stringsAsFactors'=FALSE)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: gsubfn
Loading required package: proto
Warning message:
“no DISPLAY variable so Tk is not available”Loading required package: RSQLite


### Loading CSV files

We are going to load our CSV files. 

In [2]:
# loading CSV data sets

# nba season stats 
# player data
# daily team ranking and stats 
# nba champions by season
# team id mapping data


file1 <- "../../../casestudy_Group2/Data/nba_seasons_stats.csv"
file2 <- "../../../casestudy_Group2/Data/player_data.csv"
file3 <- "../../../casestudy_Group2/Data/ranking-1.csv"
file4 <- "../../../casestudy_Group2/Data/teamid.csv"
file5 <- "../../../casestudy_Group2/Data/champs.csv"

# read CSVs
df_season <- read.csv(file1)
df_player <- read.csv(file2)
ranking_season <- read.csv(file3)
team_id <- read.csv(file4)
nba_champs <- read.csv(file5)



# Data


## NBA Ssason Stats and Player data sets

Next, we will work on cleaning and joining NBA Season Stats and Player data sets.

**NBA Season Stats**: The data-set contains aggregate individual player statistics for 67 NBA seasons. Variables include from many box-score attributes such as points, assists, rebounds, etc. The data was scraped from Basketball-reference. https://www.basketball-reference.com

**Player**:The data-set contains players data. Variables include year start, year end, age, university, etc.

In [3]:
# print top rows
head(df_season)
head(df_player)

# print columsn
colnames(df_player)
colnames(df_season)

index,Year,Player,Pos,Age,Tm,G,GS,MP,PER,⋯,FT.,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
<int>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<dbl>,⋯,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,1950,Curly Armstrong,G-F,31,FTW,63,NA,NA,NA,⋯,0.705,NA,NA,NA,176,NA,NA,NA,217,458
1,1950,Cliff Barker,SG,29,INO,49,NA,NA,NA,⋯,0.708,NA,NA,NA,109,NA,NA,NA,99,279
2,1950,Leo Barnhorst,SF,25,CHS,67,NA,NA,NA,⋯,0.698,NA,NA,NA,140,NA,NA,NA,192,438
3,1950,Ed Bartels,F,24,TOT,15,NA,NA,NA,⋯,0.559,NA,NA,NA,20,NA,NA,NA,29,63
4,1950,Ed Bartels,F,24,DNN,13,NA,NA,NA,⋯,0.548,NA,NA,NA,20,NA,NA,NA,27,59
5,1950,Ed Bartels,F,24,NYK,2,NA,NA,NA,⋯,0.667,NA,NA,NA,0,NA,NA,NA,2,4


name,year_start,year_end,position,height,weight,birth_date,college
<chr>,<int>,<int>,<chr>,<chr>,<int>,<chr>,<chr>
Alaa Abdelnaby,1991,1995,F-C,6-10,240,"June 24, 1968",Duke University
Zaid Abdul-Aziz,1969,1978,C-F,6-9,235,"April 7, 1946",Iowa State University
Kareem Abdul-Jabbar,1970,1989,C,7-2,225,"April 16, 1947","University of California, Los Angeles"
Mahmoud Abdul-Rauf,1991,2001,G,6-1,162,"March 9, 1969",Louisiana State University
Tariq Abdul-Wahad,1998,2003,F,6-6,223,"November 3, 1974",San Jose State University
Shareef Abdur-Rahim,1997,2008,F,6-9,225,"December 11, 1976",University of California


[1] "name"       "year_start" "year_end"   "position"   "height"    
[6] "weight"     "birth_date" "college"

[1] "index"  "Year"   "Player" "Pos"    "Age"    "Tm"     "G"      "GS"    
 [9] "MP"     "PER"    "TS."    "X3PAr"  "FTr"    "ORB."   "DRB."   "TRB."  
[17] "AST."   "STL."   "BLK."   "TOV."   "USG."   "OWS"    "DWS"    "WS"    
[25] "WS.48"  "OBPM"   "DBPM"   "BPM"    "VORP"   "FG"     "FGA"    "FG."   
[33] "X3P"    "X3PA"   "X3P."   "X2P"    "X2PA"   "X2P."   "eFG."   "FT"    
[41] "FTA"    "FT."    "ORB"    "DRB"    "TRB"    "AST"    "STL"    "BLK"   
[49] "TOV"    "PF"     "PTS"

## Cleaning data, formating and standardizing variables

- Formating Player name column by removing * (indicates a player is a member of NBA Hall of Fame)
- Filtering out rows with missing year and player name values
- Removing records of players who plated on multiple teams during the same season
- Standardizing NBA position names to the current

In [4]:

# fixing Player name column so we can joing the two data frames on name and birth date
# we are going to remove * (indicates a player is a member of NBA Hall of Fame)
df_season$Player <- as.character(df_season$Player)
df_season$Player <- gsub("\\*$", "", df_season$Player)

# filtering out rows that are missing year and player name values
df_season <- df_season %>% filter(!is.na(Year), !is.na(Player))

# removing rows for players who played on multiple teams in the same season (Tm = TOT)
df_season <- df_season[df_season$Tm != "TOT",]

# standardizing playing position names to the current names
df_season$Pos[df_season$Pos == "C-F"] <- "C"
df_season$Pos[df_season$Pos == "F-C"] <- "PF"
df_season$Pos[df_season$Pos == "F"] <- "PF"
df_season$Pos[df_season$Pos == "F-G"] <- "SF"
df_season$Pos[df_season$Pos == "G-F"] <- "SF"
df_season$Pos[df_season$Pos == "G"] <- "SG"

# setting position column as factor
# we will be using position column for grouping our data in further analysis
df_season$Pos <- factor(df_season$Pos, levels = c("C", "PF", "SF", "SG", "PG"))

# checking unique positions
unique(df_season$Pos)

[1] SF SG PF PG C 
Levels: C PF SF SG PG

## Creating additional variables

We are going to create NBA Player Per Game metrics for each season. We will be using these metrics in our further analysis on performance of players.

Metrics:
- **MPG**: minutes per game 
- **PPG**: points per game
- **APG**: assiste per game 
- **RPG**: rebounds per game
- **BPG**: blocks per game
- **SPG**: steals per game 
- **TOPG**: turnovers per game
- **PFPG**: personal fouls per game



In [5]:

# creating NBA Player stats Per Game for each season:
df_season <- df_season %>%
    mutate(MPG = MP/G,
           PPG = PTS/G, 
           APG = AST/G, 
           RPG = TRB/G, 
           BPG = BLK/G, 
           SPG = STL/G, 
           TOPG = TOV/G, 
           PFPG = PF/G) 


## Joining NBA season stats and Player data frames

Before we join the NBA season stats and Player data frames we will filter out seasons prior to 2002. This is because our Team ranking data starts in 2002 season.

In [6]:
# check first season in the ranking_season data set
min(substring(ranking_season$SEASON_ID, 2))

[1] "2002"

In [7]:
# filter df_season to start from 2002 season
dfseason02 <- df_season %>% filter(df_season$Year > 2001)

# droping position column in Player data set
# we will be using the standardized position (Pos) column 
df_player$position <- NULL

# joining season and player data data frames
df_season <- sqldf("select * 
                    from dfseason02 a 
                    join df_player b on a.player = name 
                    where a.Year >= year_start and a.Year <= year_end")

# check number of rows and columns
dim(df_season)

[1] 8210   66

In [8]:
# pring unique seasons
unique(df_season$Year)

[1] 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
[16] 2017

In [9]:
# pring column names
colnames(df_season)

[1] "index"      "Year"       "Player"     "Pos"        "Age"       
 [6] "Tm"         "G"          "GS"         "MP"         "PER"       
[11] "TS."        "X3PAr"      "FTr"        "ORB."       "DRB."      
[16] "TRB."       "AST."       "STL."       "BLK."       "TOV."      
[21] "USG."       "OWS"        "DWS"        "WS"         "WS.48"     
[26] "OBPM"       "DBPM"       "BPM"        "VORP"       "FG"        
[31] "FGA"        "FG."        "X3P"        "X3PA"       "X3P."      
[36] "X2P"        "X2PA"       "X2P."       "eFG."       "FT"        
[41] "FTA"        "FT."        "ORB"        "DRB"        "TRB"       
[46] "AST"        "STL"        "BLK"        "TOV"        "PF"        
[51] "PTS"        "MPG"        "PPG"        "APG"        "RPG"       
[56] "BPG"        "SPG"        "TOPG"       "PFPG"       "name"      
[61] "year_start" "year_end"   "height"     "weight"     "birth_date"
[66] "college"

In [10]:
# renaming POS column to Position
colnames(df_season)[4] <- "Position"

# checking column names
colnames(df_season)

[1] "index"      "Year"       "Player"     "Position"   "Age"       
 [6] "Tm"         "G"          "GS"         "MP"         "PER"       
[11] "TS."        "X3PAr"      "FTr"        "ORB."       "DRB."      
[16] "TRB."       "AST."       "STL."       "BLK."       "TOV."      
[21] "USG."       "OWS"        "DWS"        "WS"         "WS.48"     
[26] "OBPM"       "DBPM"       "BPM"        "VORP"       "FG"        
[31] "FGA"        "FG."        "X3P"        "X3PA"       "X3P."      
[36] "X2P"        "X2PA"       "X2P."       "eFG."       "FT"        
[41] "FTA"        "FT."        "ORB"        "DRB"        "TRB"       
[46] "AST"        "STL"        "BLK"        "TOV"        "PF"        
[51] "PTS"        "MPG"        "PPG"        "APG"        "RPG"       
[56] "BPG"        "SPG"        "TOPG"       "PFPG"       "name"      
[61] "year_start" "year_end"   "height"     "weight"     "birth_date"
[66] "college"

## Adding new variables

We are creting two additional variabls that will be used to analyze how does player's performance change as they play more seasons.

- **Player Carrier Length**: How many total years was the player in the league?
- **Season Carrier Length**: How many years was the player in the ligue during the season X?


In [11]:
# creating new variables
df_season$Playing_Career_Length <- df_season$year_end - df_season$year_start
df_season$Season_Career_Length <- df_season$Year - df_season$year_start


## Team ranking and NBA championship teams data sets

Next, we will work with Daily Team ranking and NBA championship teams data sets. 

We will aggretage the Daily Team ranking data to have the last record for each team by season.


In [12]:
# print top rows
head(ranking_season)
head(nba_champs)

TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<chr>,<chr>
1610612747,0,22019,2020-03-01,West,L.A. Lakers,59,46,13,0.780,21-7,25-6
1610612743,0,22019,2020-03-01,West,Denver,60,41,19,0.683,24-7,17-12
1610612746,0,22019,2020-03-01,West,LA Clippers,60,41,19,0.683,25-6,16-13
1610612745,0,22019,2020-03-01,West,Houston,59,39,20,0.661,21-8,18-12
1610612762,0,22019,2020-03-01,West,Utah,59,37,22,0.627,21-9,16-13
1610612760,0,22019,2020-03-01,West,Oklahoma City,60,37,23,0.617,20-12,17-11


Year,Tm
<int>,<chr>
2002,LAL
2003,SAS
2004,DET
2005,SAS
2006,MIA
2007,SAS


### Filtering data and creating new variables

Next, we will perform data cleaning on the Team Ranking data frame. We will remove the Summer season records because we are only interested in the Regular season. We are converting STANDINGSDATE variable to date format to find last record per team by season. We will also create a Season year variable to be able to join the Team Ranking data with Player Season data. 

- Filter Summer season from the Team Ranking data frame
- Converting STANDINGSDATE variable to date format
- Create Season Year variable (season_yr)

In [13]:
# excluding summer league ranking (SEASON_ID starting with 1)
# we are only interesting in the regular season
ranking_season <- sqldf("select * from ranking_season where SEASON_ID not like ('1%')")

# convert STANDINGSDATE to date so we can use the variable to find last record per team by season 
ranking_season$STANDINGSDATE <- as.Date(ranking_season$STANDINGSDATE)

# creating season year variable 
ranking_season$season_yr <- substring(ranking_season$SEASON_ID, 2)

# print unique seasons
unique(ranking_season$season_yr)


[1] "2019" "2013" "2012" "2011" "2010" "2009" "2008" "2007" "2006" "2005"
[11] "2004" "2003" "2002" "2018" "2017" "2016" "2015" "2014"

### Aggregate daily team ranking data frame

We will create a new data frame with the **ending** Team rank record for 2002 - 2019 seasons. We will aggregating Team Ranking data which provides Team rank for every day.

Columns:
- TEAM_ID: Team ID
- season_yr: season year 
- CONFERENCE: east or west conference
- TEAM: Team name
- G: games played
- W: games won
- L: games lost
- W_PCT: win % 
- HOME_RECORD: home court season record
- ROAD_RECORD: away court season record


In [14]:
# aggregate team ranking data set
# create data frame with last team standing records for each season

# create new data frame
df_rank <- sqldf("with 
                 last_date as (
                 select SEASON_ID, 
                 max(STANDINGSDATE) as last_date
                 from ranking_season
                 group by 1
                 )
                 select r.TEAM_ID, r.season_yr, r.CONFERENCE, TEAM, G, W, L, W_PCT, HOME_RECORD, ROAD_RECORD
                 from ranking_season r 
                 join last_date l on r.SEASON_ID = l.SEASON_ID
                 and r.STANDINGSDATE = l.last_date")

# print top rows
head(df_rank)

TEAM_ID,season_yr,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD
<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<chr>,<chr>
1610612747,2019,West,L.A. Lakers,59,46,13,0.780,21-7,25-6
1610612743,2019,West,Denver,60,41,19,0.683,24-7,17-12
1610612746,2019,West,LA Clippers,60,41,19,0.683,25-6,16-13
1610612745,2019,West,Houston,59,39,20,0.661,21-8,18-12
1610612762,2019,West,Utah,59,37,22,0.627,21-9,16-13
1610612760,2019,West,Oklahoma City,60,37,23,0.617,20-12,17-11


In [15]:
# checking for duplicate rows
# checking if any team has more than 1 record per season

check <- sqldf("select season_yr, TEAM, count(*) 
                from df_rank
                group by 1,2
                having count(*) > 1")

# number of duplicate rows
dim(check)

[1] 0 3

## Updating Team name abrevations

NBA Player Season (df_season) and Team ID (team_id) data frames will be joined using **Team name abrevations** and **season year** variables. 

Team ID data frame is using the current abrevation, while the Player Season data frame includes historical abrevations. We are going to update the historical Team name abrevations in the Player Season data frame to match current ones. Abrevations need to match in both data frames before we join them. 




In [16]:
# listing unique Team abrevations in the player season data frame
unique(df_season$Tm)

[1] "DEN" "DAL" "ATL" "WAS" "DET" "MIA" "MIL" "UTA" "POR" "BOS" "MEM" "NYK"
[13] "CHI" "GSW" "NJN" "ORL" "TOR" "CHH" "MIN" "SEA" "PHO" "PHI" "IND" "SAC"
[25] "SAS" "LAC" "HOU" "LAL" "CLE" "NOH" "CHA" "NOK" "OKC" "BRK" "NOP" "CHO"

In [17]:
# listing unique Team abrevations in the player season data frame
unique(team_id$ABBREVIATION)

[1] "ATL" "BOS" "NOP" "CHI" "DAL" "DEN" "HOU" "LAC" "LAL" "MIA" "MIL" "MIN"
[13] "BKN" "NYK" "ORL" "IND" "PHI" "PHX" "POR" "SAC" "SAS" "OKC" "TOR" "UTA"
[25] "MEM" "WAS" "DET" "CHA" "CLE" "GSW"

In [18]:
# checking team name abrevations that exist in NBA Player Season data frame (df_season) but not in Team ID (team_id)
# these team name abrevations will be updated to the current ones
historical_abrevations <- sqldf("select distinct Tm 
                              from df_season a 
                              left join team_id b on Tm = b.ABBREVIATION
                              where b.ABBREVIATION is null
                              order by 1")

historical_abrevations


Tm
<chr>
BRK
CHH
CHO
NJN
NOH
NOK
PHO
SEA


In [19]:

# updating team name abrevations in NBA Player Season data frame (df_season)
# i.e. Seattle SuperSonics (SEA) are Oklahoma City Thunder (OKC) 
df_season <- sqldf("select *, 
                    case when Tm = 'SEA' then 'OKC' 
                    when Tm = 'NOK' then 'NOP'
                    when Tm = 'CHO' then 'NOP'
                    when Tm = 'CHH' then 'NOP'
                    when Tm = 'NOH' then 'NOP' 
                    when Tm = 'NJN' then 'BKN'
                    when Tm = 'BRK' then 'BKN'
                    when Tm = 'PHO' then 'PHX'
                    else Tm end as Team
                    from df_season")

# droping Team (Tm) column (replaced with Team column)
df_season$Tm <- NULL

In [20]:
# checking again for missing values
check <- sqldf("select distinct Team 
                from df_season a 
                left join team_id b on Team = b.ABBREVIATION
                where b.ABBREVIATION is null")

# no missing values
check

Team
<chr>


## Joining Team Ranking, Team ID, NBA Player Season, and Championship Team data frames

We are joining aggregated **Team Ranking** and **Team ID** data frames to get the Team name abrevation variable from Team ID. Team name abrevations and season year will be used to join **Team Ranking** and **NBA Player Season**. 
Next, we will also be joining the data frame with **Championship Team** data to get the champion teams by season.


In [21]:
# join the Team Ranking and Team ID data frames 
df_rank <- sqldf("select a.*, 
                  ABBREVIATION 
                  from df_rank a 
                  join team_id b on a.TEAM_ID = b.TEAM_ID")


In [22]:
# join the Team Ranking and NBA Player Season data frames
df_season_rank <- sqldf("select a.*,
                         b.G as team_G,
                         b.W as team_W,
                         b.L as team_L,
                         b.W_PCT as team_W_PCT
                         from df_season a 
                         join df_rank b on a.Team = b.ABBREVIATION and a.Year = b.season_yr")


In [23]:
# check to make sure no records have been lost in the process 
# check number rows and columns in df_season
dim(df_season)
# check number rows and columns in new df_season_rank
dim(df_season_rank)

[1] 8210   68

[1] 8210   72

In [24]:
# join df_season_rank and Championship Team data frames
# create binary variable champ_team (0/1)
df <- sqldf("select a.*, 
             case when b.Tm is not null then 1 else 0 end as champ_team
             from df_season_rank a 
             left join nba_champs b on a.Year = b.Year and a.Team = b.Tm")

In [25]:
# check column names
colnames(df)

[1] "index"                 "Year"                  "Player"               
 [4] "Position"              "Age"                   "G"                    
 [7] "GS"                    "MP"                    "PER"                  
[10] "TS."                   "X3PAr"                 "FTr"                  
[13] "ORB."                  "DRB."                  "TRB."                 
[16] "AST."                  "STL."                  "BLK."                 
[19] "TOV."                  "USG."                  "OWS"                  
[22] "DWS"                   "WS"                    "WS.48"                
[25] "OBPM"                  "DBPM"                  "BPM"                  
[28] "VORP"                  "FG"                    "FGA"                  
[31] "FG."                   "X3P"                   "X3PA"                 
[34] "X3P."                  "X2P"                   "X2PA"                 
[37] "X2P."                  "eFG."                  "FT"                   
[40] "FTA"                   "FT."                   "ORB"                  
[43] "DRB"                   "TRB"                   "AST"                  
[46] "STL"                   "BLK"                   "TOV"                  
[49] "PF"                    "PTS"                   "MPG"                  
[52] "PPG"                   "APG"                   "RPG"                  
[55] "BPG"                   "SPG"                   "TOPG"                 
[58] "PFPG"                  "name"                  "year_start"           
[61] "year_end"              "height"                "weight"               
[64] "birth_date"            "college"               "Playing_Career_Length"
[67] "Season_Career_Length"  "Team"                  "team_G"               
[70] "team_W"                "team_L"                "team_W_PCT"           
[73] "champ_team"

In [26]:
# save data frame as CSV
write.csv(df, 'final_df.csv')

# End of Data Carpetry notebook

In [27]:
# end